## Installing the hooks
For installation, one needs to copy the hooks files from the 'hooks/' folder to the '.git/hooks/' folder. This can be done by executing the 'install-hooks.sh' Bash script.

Open Git Bash, and type in:

```bash
bash install-hooks.sh
```

To verify the installation has been successful, navigate to the '.git/hooks/' folder and check for our hook files, for example if the post-commit file is present.

On some combinations of systems or environments, after the hooks are copied, it might be necessary to run the following command to convert the Windows-style line endings (CRLF) to Unix-style line endings (LF). Otherwise the script won't execute properly, or not at all. The hooks need to be copied to the hooks folder and used before the git commits and converts the line endings itself. The line endings of the scripts need to match the environment that we're commiting from. So, for example, if the hook script was edited on a Windows system, copied to the '.git/hooks/' folder, but not treated with line endings for Linux, and then a Windows Subsystem for Linux (WSL) terminal is used to commit the files, it will throw an error about the hooks. Because the WSL expects LF endings, while the scripts have CRLF endings.

Run the following so the hooks can be executed on Unix-like systems:
```bash
# Run in a (WSL) Bash terminal
dos2unix .git/hooks/post-commit
dos2unix .git/hooks/pre-commit
```

Or, alternatively, the reverse scenario could happen, a Windows user needing to use a script that has LF endings and Git not converting them, yet.
Run the following so the hooks can be executed on Windows-style systems:
```bash
# Run in a Git Bash terminal
unix2dos .git/hooks/post-commit
unix2dos .git/hooks/pre-commit
```
This could be automated in a pre-commit hook, for the other commits. But we're not sure yet, how to solve the automation for the pre-commit hook itself.

## Pre-commit hook

The pre-commit hook is a hook that runs before a commit is made.

```bash
#!/bin/bash
# A Git pre-commit hook.


#######################
####### Messages ######
#######################

echo "HOOK - PRE-COMMIT - START:"
echo "Your commit is about to happen. Running the pre-commit hooks.."


# Display the end message
echo "HOOK - PRE-COMMIT - END"
```